In [1]:
import numpy as np
import matplotlib.pyplot as plt

from jlinops import CustomLinearOperator, BaseLinearOperator, MatrixLUInverseOperator
from scipy.sparse import issparse
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg._interface import MatrixLinearOperator

# Create a MatrixOperator

In [11]:


class MatrixOperator(BaseLinearOperator):
    """A class for representing linear operators defined by explicit matrices.
    This is a subclass of SciPy's MatrixLinearOperator, which is modified
    such that it retains its identity as a MatrixOperator after various operations
    involving other LinearOperators.
    """


    def __init__(self, A):

        super().__init__(A.dtype, A.shape)
        self.A = A
        self.__adj = None
        self.args = (A,)


    def _matrix_dot(self, x):
        """Matrix-matrix and matrix-vector multiplication. Essentially the same as
        SciPy's LinearOperator.dot().
        """
        if isinstance(x, MatrixOperator) or isinstance(x, MatrixLinearOperator):
            return self.__class__( self.A.dot(x.A) )
        elif isinstance(x, BaseLinearOperator) or isinstance(LinearOperator):
            return self.dot(x)
        elif np.isscalar(x):
            return self.__class__(x*self.A)
        else:
            if not issparse(x):
                x = np.asarray(x)
            if x.ndim == 1 or x.ndim == 2 and x.shape[1] == 1:
                return self.matvec(x)
            elif x.ndim == 2:
                return self.matmat(x)
            else:
                raise ValueError('expected 1-d or 2-d array or matrix, got %r'
                                 % x)
    


    def _matrix_rdot(self, x):
        """Matrix-matrix and matrix-vector multiplication from the right.
        """
        if isinstance(x, MatrixOperator) or isinstance(x, MatrixLinearOperator):
            return self.__class__( x.A.dot(self.A) )
        elif isinstance(x, BaseLinearOperator) or isinstance(LinearOperator):
            return self._rdot(x)
        elif np.isscalar(x):
            return self.__class__(x*self.A)
        else:
            if not issparse(x):
                # Sparse matrices shouldn't be converted to numpy arrays.
                x = np.asarray(x)

            # We use transpose instead of rmatvec/rmatmat to avoid
            # unnecessary complex conjugation if possible.
            if x.ndim == 1 or x.ndim == 2 and x.shape[0] == 1:
                return self.T.matvec(x.T).T
            elif x.ndim == 2:
                return self.T.matmat(x.T).T
            else:
                raise ValueError('expected 1-d or 2-d array or matrix, got %r'
                                 % x)



    def __call__(self, x):
        """On call.
        """
        return self*x
    


    def __mul__(self, x):
        """* multiplication.
        """
        return self._matrix_dot(x)



    def __rmul__(self,x):
        """
        """
        if np.isscalar(x):
            return self.__class__(x*self.A)
        else:
            return self._matrix_rdot(x)



    def __matmul__(self, x):
        """
        """
        if np.isscalar(x):
            raise ValueError("Scalar operands are not allowed, "
                             "use '*' instead")
        return self.__mul__(x)
    

    
    def __rmatmul__(self, x):
        """
        """
        if np.isscalar(x):
            raise ValueError("Scalar operands are not allowed, "
                             "use '*' instead")
        return self.__rmul__(x)



    def __pow__(self, p):
        """Raise to a power.
        """
        raise NotImplementedError



    def __add__(self, x):
        """Addition.
        """
        if isinstance(x, MatrixOperator) or isinstance(x, MatrixLinearOperator):
            return self.__class__(self.A + x.A)
        elif isinstance(x, BaseLinearOperator) or isinstance(LinearOperator):
            return self + x
        else:
            return NotImplemented
        


    def __neg__(self):
        """Negation.
        """
        return -1*self
    


    def __sub__(self, x):
        """Subtraction.
        """
        return self.__add__(-x)



    def _transpose(self):
        #return MatrixOperator(self.A.T)
        return self.__class__(self.A.T)
        
    T = property(_transpose)



    # def _adjoint(self):
    #     return self.__class__(self.A.H)
    
    def _adjoint(self):
        if self.__adj is None:
            self.__adj = _AdjointMatrixOperator(self)
        return self.__adj

    
    H = property(_adjoint)



    def _inv(self):
        """Creates a linear operator representing the inverse operator, which uses the LU decomposition.
        """
        return MatrixLUInverseOperator(self)

    Inv = property(_inv)




class _AdjointMatrixOperator(MatrixOperator):
    
    def __init__(self, adjoint):
        self.A = adjoint.A.T.conj()
        self.__adjoint = adjoint
        self.args = (adjoint,)
        self.shape = adjoint.shape[1], adjoint.shape[0]

    @property
    def dtype(self):
        return self.__adjoint.dtype

    def _adjoint(self):
        return self.__adjoint


In [12]:
A = np.random.normal(size=(8,5))

In [13]:
M = MatrixOperator(A)

In [14]:
M + M

<8x5 MatrixOperator with dtype=float64>

In [15]:
from scipy.sparse.linalg import aslinearoperator

In [16]:
aslinearoperator(A) + M

<8x5 _SumLinearOperator with dtype=float64>

In [18]:
M + aslinearoperator(A)

<8x5 MatrixOperator with dtype=float64>

In [17]:
aslinearoperator(A)

<8x5 MatrixLinearOperator with dtype=float64>